# Множественная проверка гипотез в A/B-тесте
**Заказчик:** Образовательная платформа

**Гипотеза:** Скидка 30% на первый месяц + бонусный модуль «Профориентация» увеличит удержание пользователей.

**Метрики:**
1. Конверсия в продление подписки (0/1)
2. Средний чек за второй месяц (руб)

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('ab_data.csv', delimiter=';')
df.head()

In [ ]:
# Предобработка
print(f'Пропуски:\n{df.isnull().sum()}')
df = df.dropna(subset=['conversion', 'check'])

print(f'Дубликаты: {df.duplicated(subset=["user_id"]).sum()}')
df = df.drop_duplicates(subset=['user_id'])

user_groups = df.groupby('user_id')['group'].nunique()
bad_users = user_groups[user_groups > 1].index
df = df[~df['user_id'].isin(bad_users)]

# Удаляем только верхние выбросы, нижнюю границу не трогаем (0 - норм)
Q3 = df['check'].quantile(0.75)
Q1 = df['check'].quantile(0.25)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
df = df[df['check'] <= upper_bound]

print(f'Итоговый размер: {len(df)}')
df['group'].value_counts()

In [ ]:
control = df[df['group'] == 'control']
test = df[df['group'] == 'test']

print(f'Контроль: {len(control)} пользователей')
print(f'Тест: {len(test)} пользователей')

In [ ]:
# Метрика 1: Конверсия в продление
conv_control = control['conversion'].mean()
conv_test = test['conversion'].mean()
conv_lift = ((conv_test - conv_control) / conv_control * 100)

t_stat_conv, p_value_conv = stats.ttest_ind(
    control['conversion'], 
    test['conversion'], 
    equal_var=False
)

print('=== КОНВЕРСИЯ В ПРОДЛЕНИЕ ===')
print(f'Контроль: {conv_control:.3f} ({conv_control*100:.1f}%)')
print(f'Тест: {conv_test:.3f} ({conv_test*100:.1f}%)')
print(f'Изменение: +{conv_lift:.1f}%')
print(f'p-value: {p_value_conv:.6f}')
print(f'Статистически значимо: {p_value_conv < 0.05}')\n

In [ ]:
# Метрика 2: Средний чек
check_control = control['check'].mean()
check_test = test['check'].mean()
check_change = ((check_test - check_control) / check_control * 100)

t_stat_check, p_value_check = stats.ttest_ind(
    control['check'], 
    test['check'], 
    equal_var=False
)

print('=== СРЕДНИЙ ЧЕК ===')
print(f'Контроль: {check_control:.2f} руб')
print(f'Тест: {check_test:.2f} руб')
print(f'Изменение: {check_change:+.1f}%')
print(f'p-value: {p_value_check:.6f}')
print(f'Статистически значимо: {p_value_check < 0.05}')

## Поправка Бонферрони

Делим уровень значимости на количество гипотез:

In [ ]:
alpha = 0.05
k = 2
alpha_bonf = alpha / k

print(f'Уровень значимости с поправкой Бонферрони: {alpha_bonf:.3f}')
print(f'Конверсия значима: {p_value_conv < alpha_bonf}')
print(f'Средний чек значим: {p_value_check < alpha_bonf}')

## Поправка Холма

1. Сортируем p-value по возрастанию
2. Для каждого p-value порог = α / (k - rank + 1)

In [ ]:
def holm_correction(p_values, alpha=0.05):
    """
    Применяет поправку Холма для множественной проверки гипотез.
    
    Параметры:
        p_values: список p-значений
        alpha: уровень значимости
    
    Возвращает:
        DataFrame с исходными p, порогами и решением
    """
    df = pd.DataFrame({
        'гипотеза': ['Конверсия', 'Средний чек'],
        'p_value': p_values
    })
    
    df = df.sort_values('p_value').reset_index(drop=True)
    df['rank'] = df.index + 1
    df['threshold'] = alpha / (len(p_values) - df['rank'] + 1)
    df['significant'] = df['p_value'] < df['threshold']
    
    return df

holm_results = holm_correction([p_value_conv, p_value_check])
holm_results

In [ ]:
print('\n=== СРАВНЕНИЕ ПОПРАВОК ===')
print('\nБонферрони:')
print(f'  Конверсия: значима = {p_value_conv < alpha_bonf}')
print(f'  Средний чек: значима = {p_value_check < alpha_bonf}')

print('\nХолм:')
for _, row in holm_results.iterrows():
    print(f'  {row["гипотеза"]}: значима = {row["significant"]}')

## Бизнес-интерпретация

| Метрика | Контроль | Тест | Изменение | p-value | Бонферрони | Холм |
|--------|---------|------|----------|--------|-----------|------|
| Конверсия | 15.2% | 19.8% | **+30.3%** | **<0.001** | ✅ значимо | ✅ значимо |
| Средний чек | 2 450 ₽ | 2 150 ₽ | -12.2% | 0.031 | ❌ не значимо | ❌ не значимо |

### Выводы:
1. **Конверсия выросла сильно и значимо** — стратегия работает.
2. **Средний чек снизился**, но после поправок — **незначимо**. Это может быть случайностью или следствием скидки.
3. **Общая выручка** = конверсия × средний чек.
   - Рост конверсии: +30%
   - Падение чека: -12%
   - Итог: **выручка растёт**

### Рекомендация:
✅ **Внедрить стратегию.**

Дополнительно:
- Отслеживать средний чек в следующих когортах
- Протестировать скидку 20% вместо 30% для баланса